## Tokenized news를 TFIDF로 벡터화

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import numpy as np
import pandas as pd
import re
import json
import os
import glob
from konlpy.tag import Okt
from collections import defaultdict
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import tqdm

In [2]:
df_tmp = pd.read_csv('../data/tokenized_news/tokenized_news_textile_okt_nouns')

In [3]:
df_tmp

,Unnamed: 0,text,date
0,0,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",2020-01-01
1,1,"노동당, 중앙, 전원, 회의, 결정, 발표, 경제, 병진, 회귀, 시사, 입장, 상...",2020-01-02
2,2,"민주화, 운동, 주기, 행진곡, 광주, 왜곡, 그날, 도청, 새벽, 방식, 진실, ...",2020-01-03
3,3,"미투, 소송, 승리, 최영미, 시인, 대한민국, 법원, 소장, 피고, 종이, 수도,...",2020-01-04
4,4,"보도, 코너, 매일, 업데이트, 서울, 세계, 최악, 도시, 한국, 외래, 관광객,...",2020-01-05
...,...,...,...
226,226,"대한의사협회, 의협, 사흘, 총파업, 돌입, 정부, 새벽, 의협, 막판, 협상, 파...",2020-08-27
227,227,"장기, 긴장, 참여, 미진, 거리, 두기, 상향, 이행, 중요, 총리, 전문가, 대...",2020-08-28
228,228,"의사, 집단, 응급환자, 병원, 제때, 인권, 단체, 정부, 의사, 집단, 대결, ...",2020-08-29
229,229,"신종, 코로나바이러스, 감염증, 코로나, 속도, 다시, 확산, 사회, 거리, 두기,...",2020-08-30


In [4]:
df_tmp.drop(columns='Unnamed: 0')

,text,date
0,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",2020-01-01
1,"노동당, 중앙, 전원, 회의, 결정, 발표, 경제, 병진, 회귀, 시사, 입장, 상...",2020-01-02
2,"민주화, 운동, 주기, 행진곡, 광주, 왜곡, 그날, 도청, 새벽, 방식, 진실, ...",2020-01-03
3,"미투, 소송, 승리, 최영미, 시인, 대한민국, 법원, 소장, 피고, 종이, 수도,...",2020-01-04
4,"보도, 코너, 매일, 업데이트, 서울, 세계, 최악, 도시, 한국, 외래, 관광객,...",2020-01-05
...,...,...
226,"대한의사협회, 의협, 사흘, 총파업, 돌입, 정부, 새벽, 의협, 막판, 협상, 파...",2020-08-27
227,"장기, 긴장, 참여, 미진, 거리, 두기, 상향, 이행, 중요, 총리, 전문가, 대...",2020-08-28
228,"의사, 집단, 응급환자, 병원, 제때, 인권, 단체, 정부, 의사, 집단, 대결, ...",2020-08-29
229,"신종, 코로나바이러스, 감염증, 코로나, 속도, 다시, 확산, 사회, 거리, 두기,...",2020-08-30


In [5]:
tfidf_vect = TfidfVectorizer(min_df=0.2, max_df=0.65)

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(df_tmp['text'])

In [6]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(df_tmp,test_size=0.2)

In [7]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 
# 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [8]:
df_tmp['cluster_label'] = cluster_label
df_tmp

,Unnamed: 0,text,date,cluster_label
0,0,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",2020-01-01,0
1,1,"노동당, 중앙, 전원, 회의, 결정, 발표, 경제, 병진, 회귀, 시사, 입장, 상...",2020-01-02,0
2,2,"민주화, 운동, 주기, 행진곡, 광주, 왜곡, 그날, 도청, 새벽, 방식, 진실, ...",2020-01-03,0
3,3,"미투, 소송, 승리, 최영미, 시인, 대한민국, 법원, 소장, 피고, 종이, 수도,...",2020-01-04,0
4,4,"보도, 코너, 매일, 업데이트, 서울, 세계, 최악, 도시, 한국, 외래, 관광객,...",2020-01-05,0
...,...,...,...,...
226,226,"대한의사협회, 의협, 사흘, 총파업, 돌입, 정부, 새벽, 의협, 막판, 협상, 파...",2020-08-27,1
227,227,"장기, 긴장, 참여, 미진, 거리, 두기, 상향, 이행, 중요, 총리, 전문가, 대...",2020-08-28,1
228,228,"의사, 집단, 응급환자, 병원, 제때, 인권, 단체, 정부, 의사, 집단, 대결, ...",2020-08-29,1
229,229,"신종, 코로나바이러스, 감염증, 코로나, 속도, 다시, 확산, 사회, 거리, 두기,...",2020-08-30,0


In [9]:
tmp_df = df_tmp

In [10]:
tmp_df[tmp_df['cluster_label']==0].sort_values(by='text')

,Unnamed: 0,text,date,cluster_label
102,102,"거대, 여당, 민주당, 민주화, 최대, 의석, 민주당, 총선, 차지, 진보, 세력,...",2020-04-17,0
12,12,"검경, 수사권, 정안, 유치원, 법, 본회의, 통과, 패트, 정국, 마무리, 정세균...",2020-01-14,0
10,10,"검찰, 대거, 좌천, 인사, 하루, 선거, 개입, 의혹, 강제, 수사, 청와대, 유...",2020-01-11,0
170,170,"검찰, 삼성, 물산, 제일모직, 합병, 과정, 불법, 혐의, 이재용, 삼성, 부회장...",2020-06-29,0
148,148,"검찰, 삼성, 물산, 제일모직, 합병, 불법, 행위, 혐의, 이재용, 삼성, 부회장...",2020-06-05,0
...,...,...,...,...
122,122,"학생, 기능, 경기, 대회, 기숙사, 빼, 눈치, 기능, 폐지, 엄포, 극단, 선택...",2020-05-08,0
105,105,"항공사, 하청, 노동자, 한숨, 대한항공, 기내, 청소노동자, 지선, 가명, 통장,...",2020-04-20,0
16,16,"해리스, 대사, 제재, 위반, 발언, 청와대, 부적절, 비판, 조치, 적용, 정부,...",2020-01-18,0
117,117,"화재, 안전, 영향, 평가, 입안, 여야, 계속, 파행, 처리, 손, 입법기관, 예...",2020-05-02,0


In [11]:
tmp_df[tmp_df['cluster_label']==1].sort_values(by='text')

,Unnamed: 0,text,date,cluster_label
68,68,"공식, 선언, 국내, 증시, 조원, 증발, 환율, 폭등, 코스피, 코스닥, 폭락, ...",2020-03-13,1
219,219,"광복절, 집회, 영향, 미반, 타이밍, 진자, 급증, 정부, 서울, 경기, 인천, ...",2020-08-20,1
59,59,"국가, 전체, 진짜, 하늘만, 전북, 군산, 아이돌, 보미, 하늘, 온통, 깜깜, ...",2020-03-04,1
75,75,"금융위기, 두, 규모, 기간, 최소, 개월, 외환, 안정, 주목, 준, 호주, 브라...",2020-03-20,1
225,225,"노벨, 경제학상, 수상자, 크루그먼, 미국, 뉴욕시립대, 석좌교수, 비롯, 국내외,...",2020-08-26,1
78,78,"뉴스, 분석, 코로, 나발, 경제, 충격, 외환위기, 상처, 전문가, 낫다, 스웨덴...",2020-03-23,1
51,51,"대구, 코로나, 전담, 병원, 진료, 전국, 민관, 의료, 집결, 치료, 지역, 대...",2020-02-25,1
73,73,"대통령, 추경, 시작, 일각, 추경, 필요성, 제기, 여야, 코로나, 확산, 대응,...",2020-03-18,1
226,226,"대한의사협회, 의협, 사흘, 총파업, 돌입, 정부, 새벽, 의협, 막판, 협상, 파...",2020-08-27,1
60,60,"동생, 기저귀, 끼니, 치매, 홀로, 노인, 안부, 전화, 거동, 장애인, 동거, ...",2020-03-05,1


In [12]:
tmp_df[tmp_df['cluster_label']==2].sort_values(by='text')

,Unnamed: 0,text,date,cluster_label
18,18,"검찰, 내부, 특수, 기획, 대결, 간부, 세력, 다툼, 본격, 해석, 문상, 기자...",2020-01-21,2
38,38,"검찰, 조국, 법무부, 장관, 부인, 정경, 교수, 재판, 교수, 일기, 공개, 교...",2020-02-12,2
34,34,"대표, 총선, 종로, 출마, 선언, 문재인, 정권, 심판, 최선, 총리, 미래, 경...",2020-02-08,2
40,40,"마음, 판정, 퇴소, 국민, 걱정, 중국, 상황, 심각, 사망, 마음, 우한, 친구...",2020-02-14,2
20,20,"문희상, 아들, 세습, 정치, 특혜, 비판, 여당, 결정, 청와대, 조국, 논쟁, ...",2020-01-23,2
37,37,"봉준호, 영화, 기생충, 올해, 미국, 아카데미, 관왕, 아카데미, 최초, 영어권,...",2020-02-11,2
44,44,"승객, 승무원, 탑승, 정부, 희망, 이송, 일본, 협의, 코로나, 확진, 부부, ...",2020-02-18,2
27,27,"신종, 코로나, 국내, 확진, 번, 환자, 접촉, 부실, 관리, 하루, 확진, 확산...",2020-02-01,2
29,29,"신종, 코로나, 확진, 정부, 제한, 입출국, 차단, 조치, 방문, 외국인, 대상,...",2020-02-03,2
19,19,"아덴만, 작전, 범위, 한시, 확대, 정부, 중동, 교민, 선박, 항행, 보호, 주...",2020-01-22,2


In [13]:
tmp_df[tmp_df['cluster_label']==3].sort_values(by='text')

,Unnamed: 0,text,date,cluster_label


In [14]:
tmp_df[tmp_df['cluster_label']==4].sort_values(by='text')

,Unnamed: 0,text,date,cluster_label


In [15]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 (리뷰)
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
tmp_df['cluster_label'] = cluster_label
tmp_df.sort_values(by='cluster_label')

,Unnamed: 0,text,date,cluster_label
0,0,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",2020-01-01,0
139,139,"위안부, 피해, 증언, 정신대, 피해자, 운동, 구분, 필요, 윤미향, 화해, 용서...",2020-05-26,0
140,140,"방역, 당국, 코로나, 어린이, 질환, 소아, 청소년, 다기, 염증, 증후군, 다기...",2020-05-27,0
141,141,"코로나, 진자, 집단, 경기, 부천시, 쿠팡, 부천, 물류, 센터, 작업장, 방역,...",2020-05-28,0
142,142,"경기, 부천시, 쿠팡, 물류, 센터, 집단, 감염, 여파, 코로나, 하루, 신규, ...",2020-05-29,0
...,...,...,...,...
20,20,"문희상, 아들, 세습, 정치, 특혜, 비판, 여당, 결정, 청와대, 조국, 논쟁, ...",2020-01-23,2
19,19,"아덴만, 작전, 범위, 한시, 확대, 정부, 중동, 교민, 선박, 항행, 보호, 주...",2020-01-22,2
18,18,"검찰, 내부, 특수, 기획, 대결, 간부, 세력, 다툼, 본격, 해석, 문상, 기자...",2020-01-21,2
44,44,"승객, 승무원, 탑승, 정부, 희망, 이송, 일본, 협의, 코로나, 확진, 부부, ...",2020-02-18,2


In [16]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['text']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [20]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('==================================================')

In [21]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=tmp_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=30 )
# top_features : 중심과 가까운 값
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['당선인', '전단', '당선자', '할머니', '등교', '김종인', '전세', '위안부', '검찰총장', '추경', '살포', '대검', '부회장', '집값', '성추행', '클럽', '유세', '김정은', '비례', '재건축', '뉴딜', '이태원', '임대', '선수', '통일부', '윤석열', '개학', '감찰', '대남', '종부세']
####### Cluster 1
Top features: ['교회', '신천지', '병상', '예배', '교인', '신도', '대구시', '콜센터', '우한', '파업', '청도', '의협', '개학', '중증', '이탈리아', '격상', '목사', '증시', '추경', '광화문', '약국', '대남', '호흡기', '내과', '아베', '음압', '코스피', '간호사', '보건소', '중국인']
####### Cluster 2
Top features: ['우한', '폐렴', '후베이성', '교민', '중국인', '전세기', '검역', '영화', '귀국', '호흡기', '사스', '보수당', '발열', '우한시', '아카데미', '메르스', '승객', '태국', '관위', '신당', '잠복기', '감찰', '공소장', '기침', '보건소', '탑승', '전염', '퇴원', '종로', '유학생']
